In [138]:
%load_ext autoreload
%autoreload 2
import imageio
from robot_painting.pytorch_planar_scene_drawing import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

import torch
cuda = torch.device('cuda')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
target_image_path = "../data/target.jpeg"
target_image_np = imageio.imread(target_image_path).astype(np.float)/256.
target_image = torch.tensor(target_image_np, device=cuda).permute(2, 0, 1)
n_channels, image_rows, image_cols = target_image.shape

print("Rows: ", image_rows)
print("Cols: ", image_cols)

brush_search_paths = ["../data/brushes/*.png"]
# Open every image in that folder
brush_paths = sum([glob.glob(sp) for sp in brush_search_paths], [])
print("Found brushes: ", brush_paths)

raw_brushes = [imageio.imread(brush_path).astype(np.float) / 256. for brush_path in brush_paths]
raw_brushes = numpy_images_to_torch(raw_brushes).cuda()

Rows:  745
Cols:  1280
Found brushes:  ['../data/brushes/dry-brush-stroke-21.png']


In [ ]:
# Evaluate how long it takes to apply a brush to an image.
current_image = torch.ones(1, 3, image_rows, image_cols).cuda()
target_image_oil = b2p(target_image)
b2p, p2b = oilpaint_converters(device=torch.device('cuda'))

N_sprites_in_batch = 10
orig_brush_mask = raw_brushes[0][3, :].cuda()
brushes = torch.empty(N_sprites_in_batch, 4, orig_brush_mask.shape[0], orig_brush_mask.shape[1], device=cuda)
times = {
    "brush": [],
    "drawing": [],
    "oil": []
}
    
N_iters = 10000
for iter_k in tqdm(range(N_iters)):
    # Draw brush at random pose with random color.
    start_time = time.time()
    
    sprite_poses = torch.stack([
        torch.randint(low=int(-image_rows / 2), high=int(image_rows / 2), size=(N_sprites_in_batch,)),
        torch.randint(low=int(-image_cols / 2), high=int(image_cols / 2), size=(N_sprites_in_batch,)),
        torch.rand(N_sprites_in_batch) * np.pi * 2.
    ]).T
    # Random color + opacity.
    for k in range(N_sprites_in_batch):
        center = sprite_poses[k, 0:2]
        center_x = int(min(max(center[0].item() + (image_rows / 2), 0), image_rows))
        center_y = int(min(max(center[1].item() + (image_cols / 2), 0), image_cols))
        for i in range(3):
            brushes[k, i, :, :] = target_image[i, center_x, center_y]
        brushes[k, 3, :, :] = orig_brush_mask * torch.rand(1, device=cuda)
    post_brush_setup_time = time.time()
    
    sprites_scales = torch.ones(N_sprites_in_batch, 1) * (1. - float(iter_k) / N_iters)**2.
    sprite_ims = draw_sprites_at_poses(
          sprite_poses, sprites_scales,
          brushes.shape[2], brushes.shape[3],
          image_rows, image_cols,
          brushes
    )
    
    post_drawing_time = time.time()
    
    # How much does rendering each one get us closer
    # to the desired image?
    image_pre_oil = b2p(current_image)
    sprite_ims_oil = b2p(sprite_ims)
    
    curr_error = torch.mean(torch.square(target_image_oil - image_pre_oil[:3, :, :]))
    alphas = 1. - sprite_ims_oil[:, 3:, :, :]  # alphas inverted in absorb space
    inv_alphs = sprite_ims_oil[:, 3:, :, :]
        
    # Reduce down to one image
    # Permute for easier alpha combo
    image = image_pre_oil[0, :3, :, :]
    scaled_sprites = sprite_ims_oil[:, :3, :, :] * alphas
    for k in range(sprite_ims_oil.shape[0]):
        new_image = image * inv_alphs[k, ...] + scaled_sprites[k, ...]
        new_error = torch.mean(torch.square(target_image_oil - new_image))
        if new_error <= curr_error:
            image = new_image
            
    current_image = p2b(image).unsqueeze(0)
    
    end_time = time.time()
    
    times["brush"].append(post_brush_setup_time - start_time)
    times["drawing"].append(post_drawing_time - post_brush_setup_time)
    times["oil"].append(end_time - post_drawing_time)
    
sizes = [np.mean(val)*1000 for val in times.values()]
total_times = sum([np.array(val) for val in times.values()])
labels = ["%s (%0.2f +/- %f ms)" % (key, np.mean(val)*1000, np.std(val)*1000) for key, val in times.items()]

plt.figure()
plt.subplot(2, 1, 1)
plt.pie(sizes, labels=labels, normalize=True)
plt.subplot(2, 1, 2)
plt.plot(total_times)
plt.title("Total times across iters")

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
dpi = 300
plt.figure(dpi=dpi).set_size_inches(image_rows/dpi, image_cols/dpi)
plt.imshow(target_image.permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.title("Target")

dpi = 300
plt.figure(dpi=dpi).set_size_inches(image_rows/dpi, image_cols/dpi)
plt.imshow(current_image[0, :].permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.title("Recovered")